## Bibioteki + funkcje

In [1]:
%run ../scripts/init0.py

Wczytano podstawowe bilioteki!


In [2]:
%run ../scripts/my_functions_01.py

Wczytano funkcje użytkownika!


In [3]:
%run ../scripts/load_data.py 'raw'

Zmienne: train + test. Pomyślnie wczytano dane surowe!


## Oczyszczanie danych

Z praktycznego punktu widzenia łączę train i test w jeden zbiór. Podział jest potrzebny dopiero na etapie modelowania.

**data = train + test**

In [4]:
data = train.append(test).sort_index()

In [5]:
len(data) == len(train) + len(test)

True

Aktualne zużycie pamięci

In [6]:
print('data: ' + str(np.round(data.memory_usage().sum()/1024/1024, 2)) + ' MB')

data: 38.99 MB


Reset index

In [7]:
data = data.reset_index()

Nowe nazwy zmiennych (do prostszego grupowania)

In [8]:
new_names = {
    'b_request_date':           'date_request_b', 
    'date_confirmed':           'date_confirmed',
    'conract_open_date':        'date_contract_open',
    'final_payment_date':       'date_final_payment',
    'real_date_closed_account': 'date_closed_account',
    'str_start':                'date_start_sth',
    'cred_limit':               'val_credit_limit',
    'cur_debt':                 'val_debt_curr',
    'next_payment':             'val_payment_next',
    'cur_balance':              'val_balance_curr',
    'cur_overdue_debt':         'val_debt_overdue_curr',
    'max_amount_debt':          'val_debt_amount_max',
    'customer_id':              'id_customer',
    'b_id':                     'code_b',
    'contract_type':            'code_contract_type',
    'contract_status':          'code_contract_status',
    'cur':                      'code_currency',
    'trustability_code':        'code_trustability',
    'interest_rate_loan':       'loan_rate',
    'index':                    'id_index',
}

In [9]:
data = data.rename(columns=new_names)
data = data[sorted(data.columns)]
data.columns

Index(['cnt_delay_30d_59d', 'cnt_delay_5d_29d', 'cnt_delay_60d_89d',
       'cnt_delay_upto_30d', 'cnt_delay_upto_5d', 'cnt_delay_upto_90d',
       'code_b', 'code_contract_status', 'code_contract_type', 'code_currency',
       'code_frequency_payments', 'code_relationship_contract',
       'code_trustability', 'date_closed_account', 'date_confirmed',
       'date_contract_open', 'date_final_payment', 'date_request_b',
       'date_start_sth', 'duration_debt_days', 'id_customer', 'id_index',
       'is_bad', 'loan_rate', 'val_balance_curr', 'val_credit_limit',
       'val_debt_amount_max', 'val_debt_curr', 'val_debt_overdue_curr',
       'val_payment_next'],
      dtype='object')

Usunięcie wartości niezgodnej z typem kolumny

In [10]:
data.loc[data.date_closed_account == '8/22/1899','date_closed_account'] = np.nan

Zamiana przecinków na kropki

In [11]:
data['val_debt_curr'] = data['val_debt_curr'].apply(lambda x: x if isinstance(x, float) else str(x.replace(',','.')))
data['val_payment_next'] = data['val_payment_next'].apply(lambda x: x if isinstance(x, float) else str(x.replace(',','.')))
data['val_debt_overdue_curr'] = data['val_debt_overdue_curr'].apply(lambda x: x if isinstance(x, float) else str(x.replace(',','.')))
data['val_debt_amount_max'] = data['val_debt_amount_max'].apply(lambda x: x if isinstance(x, float) else str(x.replace(',','.')))

Problem z formatem zapisu code_frequency_payments, dlatego zamiana A i B na 8 i 9

In [12]:
data.code_frequency_payments.unique()

array([4.0, '3', '7', 7.0, 3.0, '0', '2', 2.0, 6.0, nan, 0.0, '6', '5',
       'A', 5.0, '4', '1', 1.0, 'B'], dtype=object)

In [13]:
data.loc[data.code_frequency_payments == "A", 'code_frequency_payments'] = '8'
data.loc[data.code_frequency_payments == "B", 'code_frequency_payments'] = '9'

In [14]:
data.code_frequency_payments.unique()

array([4.0, '3', '7', 7.0, 3.0, '0', '2', 2.0, 6.0, nan, 0.0, '6', '5',
       '8', 5.0, '4', '1', 1.0, '9'], dtype=object)

Optymalizacja pamięci

In [15]:
for col in data.columns:
    tmp = detect_col_type(data[col])
    if tmp != data[col].dtype:
        data[col] = data[col].astype(tmp)

cnt_delay_30d_59d(int64): wykryto bardziej odpowiedni typ: uint8
cnt_delay_5d_29d(int64): wykryto bardziej odpowiedni typ: uint8
cnt_delay_60d_89d(int64): wykryto bardziej odpowiedni typ: uint8
cnt_delay_upto_30d(int64): wykryto bardziej odpowiedni typ: uint8
cnt_delay_upto_5d(int64): wykryto bardziej odpowiedni typ: uint8
cnt_delay_upto_90d(int64): wykryto bardziej odpowiedni typ: uint8
code_b(int64): wykryto bardziej odpowiedni typ: uint8
code_contract_status(int64): wykryto bardziej odpowiedni typ: uint8
code_contract_type(int64): wykryto bardziej odpowiedni typ: uint8
code_currency(int64): wykryto bardziej odpowiedni typ: uint8
code_frequency_payments(object): wykryto bardziej odpowiedni typ: float64
code_relationship_contract(int64): wykryto bardziej odpowiedni typ: uint8
code_trustability(object): typ object jest OK! - znaleziono znaki nie będące liczbami ani kropką!
date_closed_account(object): wykryto bardziej odpowiedni typ: float64
date_confirmed(int64): wykryto bardziej odpo

Zużycie pamięci po konwersjach

In [16]:
print('data: ' + str(np.round(data.memory_usage().sum()/1024/1024, 2)) + ' MB')

data: 20.31 MB


In [17]:
for col in data.columns:
    tmp = detect_col_type(data[col])

cnt_delay_30d_59d(uint8): typ uint8 jest OK!
cnt_delay_5d_29d(uint8): typ uint8 jest OK!
cnt_delay_60d_89d(uint8): typ uint8 jest OK!
cnt_delay_upto_30d(uint8): typ uint8 jest OK!
cnt_delay_upto_5d(uint8): typ uint8 jest OK!
cnt_delay_upto_90d(uint8): typ uint8 jest OK!
code_b(uint8): typ uint8 jest OK!
code_contract_status(uint8): typ uint8 jest OK!
code_contract_type(uint8): typ uint8 jest OK!
code_currency(uint8): typ uint8 jest OK!
code_frequency_payments(float64): typ float64 jest OK! - na razie nie ruszam float64
code_relationship_contract(uint8): typ uint8 jest OK!
code_trustability(object): typ object jest OK! - znaleziono znaki nie będące liczbami ani kropką!
date_closed_account(float64): typ float64 jest OK! - na razie nie ruszam float64
date_confirmed(uint16): typ uint16 jest OK!
date_contract_open(uint16): typ uint16 jest OK!
date_final_payment(float64): typ float64 jest OK! - na razie nie ruszam float64
date_request_b(uint16): typ uint16 jest OK!
date_start_sth(float64): t

In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170335 entries, 0 to 170334
Data columns (total 30 columns):
cnt_delay_30d_59d             170335 non-null uint8
cnt_delay_5d_29d              170335 non-null uint8
cnt_delay_60d_89d             170335 non-null uint8
cnt_delay_upto_30d            170335 non-null uint8
cnt_delay_upto_5d             170335 non-null uint8
cnt_delay_upto_90d            170335 non-null uint8
code_b                        170335 non-null uint8
code_contract_status          170335 non-null uint8
code_contract_type            170335 non-null uint8
code_currency                 170335 non-null uint8
code_frequency_payments       170210 non-null float64
code_relationship_contract    170335 non-null uint8
code_trustability             164826 non-null object
date_closed_account           84244 non-null float64
date_confirmed                170335 non-null uint16
date_contract_open            170335 non-null uint16
date_final_payment            159280 non-null float

### ZAPIS

In [19]:
data.to_hdf('../data/data.base.h5', 'base', format='table', mode='w')
print('Zapisano data do pliku ../data/data.base.h5')

Zapisano data do pliku ../data/data.base.h5
